In [39]:
# platform libraries
import openeo
from sentinelhub import (SHConfig, SentinelHubRequest, DataCollection, MimeType, CRS, BBox, bbox_to_dimensions, geometry)

# utility libraries
from datetime import date
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import xarray as xr

In [40]:
config = SHConfig()
config.sh_client_id = %env SH_CLIENT_ID
config.sh_client_secret = %env SH_CLIENT_SECRET

In [41]:
conn = openeo.connect('https://jjdxlu8vwl.execute-api.eu-central-1.amazonaws.com/production')

In [42]:
conn = conn.authenticate_basic(username=config.sh_client_id, password=config.sh_client_secret)

In [43]:
#conn.describe_account()

In [44]:
#env

In [45]:
catchment_outline = gpd.read_file('data/catchment_outline.geojson') # geopandas not accepted in pg, try shapely
catchment_outline_json = catchment_outline.to_json()
bbox = catchment_outline.bounds.iloc[0] # for load_collection, maybe replace by json later

In [46]:
collection      = 'SENTINEL2_L2A_SENTINELHUB'
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':4326}
#spatial_extent  = {'west':11.020833,'east':11.366667,'south':46.653599,'north':46.954167,'crs':4326}
#spatial_extent  = {'west':11.0,'east':11.1,'south':46.6,'north':46.7,'crs':4326}
#spatial_extent  = {'west':11.08,'east':11.11,'south':46.77,'north':46.79,'crs':4326}
#temporal_extent = ["2018-02-01", "2018-06-30"]
temporal_extent = ["2018-02-01", "2018-02-15"]
#bands           = ['B03', 'B11', 'CLM'] # ['B02', 'B03', 'B04', 'CLM']
bands           = ['B03', 'B11'] # ['B02', 'B03', 'B04', 'CLM']

In [47]:
s2 = conn.load_collection(collection,
                          spatial_extent=spatial_extent, # put json here
                          bands=bands,
                          temporal_extent=temporal_extent)

In [48]:
green = s2.band("B03")
swir = s2.band("B11")
ndsi = (green - swir) / (green + swir)
#ndsi = s2.filter_bands("B03", "B11").reduce_dimension(dimension="bands",reducer = )
ndsi

In [50]:
#ndsi_mean = ndsi.reduce_dimension(dimension="t", reducer = "mean")
ndsi_max = ndsi.max_time()
ndsi_max

In [51]:
# needs read_vector()
#ndsi_agg = ndsi_max.aggregate_spatial(geometries = catchment_outline_json, reducer = "mean") # needs read_vector as well, strange.
# needs read_vector()
ndsi_agg = ndsi_max.aggregate_spatial(geometries = 'https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/exercises/data/catchment_outline.geojson', reducer = "mean")
# reduce_spatial, not available in the python client, doesn't take geometry into account
#ndsi_agg = ndsi_max.reduce_spatial(reducer = "mean")
ndsi_agg

In [ ]:
ndsi_agg.print_json()

{
  "process_graph": {
    "loadcollection1": {
      "process_id": "load_collection",
      "arguments": {
        "bands": [
          "B03",
          "B11"
        ],
        "id": "SENTINEL2_L2A_SENTINELHUB",
        "spatial_extent": {
          "west": 11.020833333333357,
          "east": 11.366666666666694,
          "south": 46.653599378797765,
          "north": 46.954166666666694,
          "crs": 4326
        },
        "temporal_extent": [
          "2018-02-01",
          "2018-02-15"
        ]
      }
    },
    "reducedimension1": {
      "process_id": "reduce_dimension",
      "arguments": {
        "data": {
          "from_node": "loadcollection1"
        },
        "dimension": "bands",
        "reducer": {
          "process_graph": {
            "arrayelement1": {
              "process_id": "array_element",
              "arguments": {
                "data": {
                  "from_parameter": "data"
                },
                "index": 0
             

In [57]:
ndsi_agg.save_result(format = "json")

In [58]:
ndsi_agg.print_json()

{
  "process_graph": {
    "loadcollection1": {
      "process_id": "load_collection",
      "arguments": {
        "bands": [
          "B03",
          "B11"
        ],
        "id": "SENTINEL2_L2A_SENTINELHUB",
        "spatial_extent": {
          "west": 11.020833333333357,
          "east": 11.366666666666694,
          "south": 46.653599378797765,
          "north": 46.954166666666694,
          "crs": 4326
        },
        "temporal_extent": [
          "2018-02-01",
          "2018-02-15"
        ]
      }
    },
    "reducedimension1": {
      "process_id": "reduce_dimension",
      "arguments": {
        "data": {
          "from_node": "loadcollection1"
        },
        "dimension": "bands",
        "reducer": {
          "process_graph": {
            "arrayelement1": {
              "process_id": "array_element",
              "arguments": {
                "data": {
                  "from_parameter": "data"
                },
                "index": 0
             

In [53]:
ndsi_agg.download("ndsi_agg.json")

OpenEoApiError: [400] 400: Invalid process graph: Process 'read_vector' could not be found in the list of processes.

In [54]:
pth_res = "ndsi_agg.json"
res = json.load(pth_res)
res

AttributeError: 'str' object has no attribute 'read'

In [55]:
# batch job
ndsi_agg_fin = ndsi_agg.save_result(format="json")
ndsi_agg_fin_job = ndsi_agg_fin.create_job(title = "ndsi_max")
ndsi_agg_fin_job.start_job()

OpenEoApiError: [400] 400: Invalid process graph: Process 'read_vector' could not be found in the list of processes.